# Dogs vs Cats Image Classification using CNN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Unzipping the Cats_Dogs Data
#!unzip "/content/drive/MyDrive/Cats_Dogs_Data.zip" -d "/content/drive/MyDrive/Cats_Dogs_Data"

### Importing the keras libraries and packages

In [ ]:
#Helps in initialising the neural network with sequence of layers (another method is through graphs)
from keras.models import Sequential

#Helps in initialising the convolutional layer 
# 2D because of images and 3D for videos(contains time dimension)
from keras.layers import Convolution2D

#Helps to add pooling layers
from keras.layers import MaxPooling2D

#Converts pooled features(obtained from Convolutional2D,MaxPooling2D) into a large feature vector 
#that becomes input for fully connected layers
from keras.layers import Flatten

#Connects densely layers to form Neural Network
from keras.layers import Dense

### Initializing the CNN

In [ ]:
classifier = Sequential()

# 1. Convolutional Layer
#(32,3,3) -> (No.of feature detectors, no. of rows, no. of columns in feature detector)
#input_shape -> Shape of image where the feature detector will be applied (64 pixels, 64px, 3 color channels)
#activation -> purpose here is different, its not handling the weights instead, fix the values of pixels to not go to negative
classifier.add(Convolution2D(32,3,3, input_shape = (128,128,3) , activation = 'relu'))

# 2. Pooling Layer
#pool_size -> Matrix to take values from the feature map
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Adding 2nd Convolutional Layer (they two together when we add convolutional layer)
#We don't need to add input_shape as Keras will automatically give input from the previous pooled layer
classifier.add(Convolution2D(32,3,3 , activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

# 3. Flattening Layer
#Classifier will automatically know that the output of above will be passed to flattening layer, so no need to add any parameter
classifier.add(Flatten())

# 4. Fully connected Layer
#Hidden layer
#units defining no. of neurons in hidden layer (no thumb rule to decide, need to experiment)
classifier.add(Dense(units = 128 , activation = 'relu'))
#Output layer
#unitss as 1, we just need to predict whether cat or dog
classifier.add(Dense(units = 1 , activation = 'sigmoid'))


### Compiling the CNN

In [ ]:
#These parameters define we are going for Stochastic Gradient Descent Algorithm
#Loss = 'binary_crossentropy' if binary classification , if multiclass then 'categorical_crossentropy'
classifier.compile(optimizer = 'adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])

### Summary of Model

In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 42, 42, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               36992     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

### Image Augmentation

In [ ]:
#Image Data Generator to generate images
from keras.preprocessing.image import ImageDataGenerator

#Image augmentation options
train_datagen = ImageDataGenerator(
        rescale=1./255,  #Scales all the image pixesl b/w 0 and 1
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255) #in Testing dataset, just the rescaling of images we need

#Training on the images by taking them from directory
training_set = train_datagen.flow_from_directory(
                              #Path of training set images
                            'drive/MyDrive/Cats_Dogs_Data/Cats_Dogs_Data/training_set',
                            #Shape of the image we want to keep, so that all images are in exact shape
                            target_size=(128,128),
                            #Random batch of images coming to be trained
                            batch_size=32,
                            #Classes to be trained , if multiclass then 'category'
                            class_mode='binary')

#Same as above (now for testing data)
test_set = test_datagen.flow_from_directory(
                                'drive/MyDrive/Cats_Dogs_Data/Cats_Dogs_Data/test_set',
                                target_size=(128,128),
                                batch_size=32,
                                class_mode='binary')

#Fitting through the CNN model
batch_size=32
classifier.fit(
        training_set,
        #No. of images in training set  / Batch size
        steps_per_epoch=250,              #steps_per_epoch * batch_size = number_of_rows_in_train_data
        #No. of epoch
        epochs=25,
        #Validates using test_set
        validation_data=test_set,
        #No. of images in test set / Batch Size
        validation_steps=63)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 2921s 12s/step - loss: 0.6868 - accuracy: 0.5451 - val_loss: 0.6399 - val_accuracy: 0.6620
Epoch 2/25
250/250 [==============================] - 58s 232ms/step - loss: 0.6472 - accuracy: 0.6224 - val_loss: 0.5983 - val_accuracy: 0.6890
Epoch 3/25
250/250 [==============================] - 57s 228ms/step - loss: 0.5975 - accuracy: 0.6832 - val_loss: 0.5889 - val_accuracy: 0.6875
Epoch 4/25
250/250 [==============================] - 57s 228ms/step - loss: 0.5811 - accuracy: 0.6902 - val_loss: 0.5513 - val_accuracy: 0.7225
Epoch 5/25
250/250 [==============================] - 57s 227ms/step - loss: 0.5525 - accuracy: 0.7163 - val_loss: 0.5682 - val_accuracy: 0.7175
Epoch 6/25
250/250 [==============================] - 57s 227ms/step - loss: 0.5443 - accuracy: 0.7191 - val_loss: 0.5310 - val_accuracy: 0.7415
Epoch 7/25
250/250 [==========================

Model achieved an training accuracy of 80 % with validation accuracy of 79%.